In [2]:
import requests
from bs4 import BeautifulSoup

URL = 'https://www.citilink.ru/catalog/ssd-nakopiteli/'

URL = requests.head(URL, allow_redirects=True).url

#print(URL)

html1 = requests.get(URL).text

products_list = BeautifulSoup(html1).find('div', {'class': 'ProductCardCategoryList__grid-container'})

#products_list = BeautifulSoup(html1).findAll('div', {'class': 'product_data__gtm-js product_data__pageevents-js  ProductCardVertical js--ProductCardInListing ProductCardVertical_normal ProductCardVertical_shadow-hover ProductCardVertical_separated'})

#print(products_list)

html1 = requests.get(URL)
html1

<Response [200]>

In [3]:
def URLsFromPage(base_url = 'https://www.citilink.ru/catalog/ssd-nakopiteli/'):
    
     #функция предназначена для извлечения ссылок на каждый товар в опеределенной категории товаров
    
    
    links = [] #список для ссылок на каждый товар
    
    for page in range (1, getPagesAmount(requests.get(base_url).text)+1, 1):
        
        URL = base_url + '?p=' + str(page)
        
        html = requests.get(URL).text
    
        products_list = BeautifulSoup(html).findAll('a',
                                        {'class': 'ProductCardVertical__name Link js--Link Link_type_default'})

        for product in products_list:
            links.append('https://www.citilink.ru' + BeautifulSoup.get(product, 'href') + 'properties/')
    
    return links
    
    
def getPagesAmount(html1):
    
    pages_amount = BeautifulSoup(html1).find('a', {'class': 'PaginationWidget__page js--PaginationWidget__page PaginationWidget__page_last PaginationWidget__page-link'})

    pages_amount = BeautifulSoup.get(pages_amount, 'data-page')

    pages_amount = int(pages_amount)
    
    return pages_amount

print(0)

0


In [4]:

SSD_LIST = URLsFromPage()
    

In [6]:
SSD_LINKS_LIST = SSD_LIST

len(SSD_LINKS_LIST)


493

In [29]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import time

URL = 'https://www.citilink.ru/product/ssd-nakopitel-a-data-ultimate-su650-asu650ss-240gt-r-240gb-2-5-sata-ii-1091578/properties/'


html = requests.get(URL).text

class ssd():
    def __init__(self, num, vol, speed, price):
        self.num = num
        self.vol = vol
        self.speed = speed
        self.price = price
        
    def __str__(self):
        return str({'num': str(self.num), 'vol': str(self.vol), 'speed': str(self.speed), 'price': str(self.price)})



def getID(URL):
    html_f = requests.get(URL).text
    num0 = BeautifulSoup(html_f).find('div', {'class': 'ProductPageUsersActivity__product-id'})
    #print(type(num0))
    #print(num0)
    if isinstance(num0, bs4.element.Tag) == False:
        while isinstance(num0, bs4.element.Tag) == False:
            s=get_tor_session()
            html_f = s.get(URL).text
            renew_connection()
            num0 = BeautifulSoup(html_f).find('div', {'class': 'ProductPageUsersActivity__product-id'})
    #print('after:')
    #print(type(num0))
    #print(num0)
        

    num = BeautifulSoup.getText(num0)

    num = num.strip()

    num2 = re.split(': ', num)
    num = num2[1]
    return num

def getVolume(URL):
    s=get_tor_session()
    html_f = s.get(URL).text
    renew_connection()
    vol = ''
    vol_list = BeautifulSoup(html_f).findAll('div', {'class': 'Specifications__row'})
    for v in vol_list:
        match = re.search('Объем накопителя', str(v))
        if match:
            vol = v
            
            
    vol = BeautifulSoup(str(vol)).find('div', {'class': 'Specifications__column Specifications__column_value'})
    
    if isinstance(vol, bs4.element.Tag) == False:
        while isinstance(vol, bs4.element.Tag) == False:
            s=get_tor_session()
            html_f = s.get(URL).text
            renew_connection()
            vol_list = BeautifulSoup(html_f).findAll('div', {'class': 'Specifications__row'})
            for v in vol_list:
                match = re.search('Объем накопителя', str(v))
                if match:
                    vol = v
            vol = BeautifulSoup(str(vol)).find('div', {'class': 'Specifications__column Specifications__column_value'})
    
    
    
    vol2 = BeautifulSoup.getText(vol)
    
    
    vol2 = vol2.strip()
    vol2 = re.split(' ', vol2)
    vol = int(vol2[0])
    return vol

def getSpeed(URL):
    
    s=get_tor_session()
    print(1)
    html_f = s.get(URL, allow_redirects=True).text
    print(s.get(URL, allow_redirects=True))

    renew_connection()
    vol = ''
    vol_list = BeautifulSoup(html_f).findAll('div', {'class': 'Specifications__row'})
    #print(type(vol_list))
    for v in vol_list:
        match = re.search('Максимальная скорость чтения', str(v))
        if match:
            vol = v
            
            
    vol = BeautifulSoup(str(vol)).find('div', {'class': 'Specifications__column Specifications__column_value'})
    print(2)
    #print(html_f)
    if isinstance(vol, bs4.element.Tag) == False:
        while isinstance(vol, bs4.element.Tag) == False:
            time.sleep(1)
            html_f = s.get(URL, allow_redirects=True).text
            renew_connection()
            vol_list = BeautifulSoup(html_f).findAll('div', {'class': 'Specifications__row'})
            #print(type(vol_list))
            for v in vol_list:
                match = re.search('Максимальная скорость чтения', str(v))
                if match:
                    vol = v
            vol = BeautifulSoup(str(vol)).find('div', {'class': 'Specifications__column Specifications__column_value'})
    
    print(3)
    vol2 = BeautifulSoup.getText(vol)
    vol2 = vol2.strip()
    vol2 = re.split(' ', vol2)
    vol = int(vol2[0])
    return vol
    
def getPrice(URL):
    s=get_tor_session()
    html_f = s.get(URL).text
    renew_connection()
    price = BeautifulSoup(html_f).find('span', {'class': 'ProductPagePriceSection__default-price_current-price js--ProductPagePriceSection__default-price_current-price ProductPagePriceSection__default-price-value'})
    if isinstance(price, bs4.element.Tag) == False:
        while isinstance(price, bs4.element.Tag) == False:
            s=get_tor_session()
            html_f = s.get(URL).text
            renew_connection()
            price = BeautifulSoup(html_f).find('span', {'class': 'ProductPagePriceSection__default-price_current-price js--ProductPagePriceSection__default-price_current-price ProductPagePriceSection__default-price-value'})
    
    price2 = BeautifulSoup.getText(price)
    
    return int(price2)


getSpeed(URL)

1
<Response [429]>
2


KeyboardInterrupt: 

In [16]:
html_f

NameError: name 'html_f' is not defined

In [ ]:
import requests
from stem.control import Controller
from stem import Signal
import tqdm
from progress.bar import IncrementalBar
import time



def get_tor_session():
    # инициализировать сеанс запросов
    session = requests.Session()
    # установка прокси для http и https на localhost: 9050
    # для этого требуется запущенная служба Tor на вашем компьютере и прослушивание порта 9050 (по умолчанию)
    session.proxies = {"http": "socks5://localhost:9050", "https": "socks5://localhost:9050"}
    return session

def renew_connection():
    with Controller.from_port(port=9051) as c:
        c.authenticate()
        # отправить сигнал NEWNYM для установления нового чистого соединения через сеть Tor
        c.signal(Signal.NEWNYM)
        #time.sleep(5)

#bar = IncrementalBar('IP testing', max = 10)
for n in range(10):
    #bar.next()
    #renew_connection()
    s = get_tor_session()
    ip = s.get("http://icanhazip.com").text
    print("IP:", ip)
    renew_connection()
    s = get_tor_session()
    ip = s.get("http://icanhazip.com").text
    print("IP:", ip)
    #time.sleep(1)
    print('\n iteration end \n')
    #time.sleep(5)
    renew_connection()

#bar.finish()

Массив с ссылками получен. Функции для получения данных со страницы по каждой ссылке есть. Класс для описания ssd есть. Пора это всё обработать!

In [10]:
from ipywidgets import IntProgress
from IPython.display import display
import csv

prgBar = IntProgress(min = 0, max = 493) # Создаем прогрессбар
display(prgBar)

SSD_LIST = []

with open("ssds.csv", mode="w", encoding='utf-8') as w_file:
    for link in SSD_LINKS_LIST:
        
        
        id_s = getID(link)
        
        vol = getVolume(link)
        
        speed = getSpeed(link)
        
        price = getPrice(link)
        SSD_LIST.append(ssd(id_s, vol, speed, price))
    
    
        file_writer = csv.writer(w_file, delimiter = ";", lineterminator="\r")
        file_writer.writerow([id_s, vol, speed, price])
    
    

        prgBar.value += 1
        

SSD_LIST

IntProgress(value=0, max=499)

NameError: name 'SSD_LINKS_LIST' is not defined

In [26]:

ssd_listik = []
with open("ssds.csv", encoding='utf-8') as r_file:
    # Создаем объект reader, указываем символ-разделитель ","
    file_reader = csv.reader(r_file, delimiter = ";")
    # Считывание данных из CSV файла
    for row in file_reader:
        ssd_listik.append(ssd(row[0], row[1], row[2], row[3]))
    
    
str(ssd_listik)



'[<__main__.ssd object at 0x0000022F1A8425B0>, <__main__.ssd object at 0x0000022F1A842040>, <__main__.ssd object at 0x0000022F1A842BE0>, <__main__.ssd object at 0x0000022F1A8423A0>, <__main__.ssd object at 0x0000022F1A842190>, <__main__.ssd object at 0x0000022F1A842D00>, <__main__.ssd object at 0x0000022F155CE6D0>, <__main__.ssd object at 0x0000022F155CE370>, <__main__.ssd object at 0x0000022F155CE400>, <__main__.ssd object at 0x0000022F155CE4C0>, <__main__.ssd object at 0x0000022F155CE2E0>, <__main__.ssd object at 0x0000022F155CE160>, <__main__.ssd object at 0x0000022F155CE1F0>, <__main__.ssd object at 0x0000022F155CE1C0>, <__main__.ssd object at 0x0000022F155CEDF0>, <__main__.ssd object at 0x0000022F155CE970>, <__main__.ssd object at 0x0000022F155CEC40>, <__main__.ssd object at 0x0000022F155CED60>, <__main__.ssd object at 0x0000022F155CE580>, <__main__.ssd object at 0x0000022F155CE520>, <__main__.ssd object at 0x0000022F155CE070>, <__main__.ssd object at 0x0000022F155CE700>, <__main_

In [13]:
import requests
import random
from bs4 import BeautifulSoup as bs

def get_free_proxies():
    url = "https://free-proxy-list.net/"
    # получаем ответ HTTP и создаем объект soup
    #soup = bs(requests.get(url).content, "html.parser")
    print(requests.get(url))
    html = requests.get(url).text
    #print(html)
    soup = bs(html)
    proxies = []
    for row in bs(html).find("table", attrs={"class": "table table-striped table-bordered"}).find_all("tr")[1:]:
        tds = row.find_all("td")
        try:
            ip = tds[0].text.strip()
            port = tds[1].text.strip()
            host = f"{ip}:{port}"
            proxies.append(host)
        except IndexError:
            continue
    return proxies

free_proxies = get_free_proxies()
"""
print(f'Обнаружено бесплатных прокси - {len(free_proxies)}:')
for i in range(len(free_proxies)):
    print(f"{i+1}) {free_proxies[i]}")
    
    """
def get_session(proxies):
    # создать HTTP‑сеанс
    session = requests.Session()
    # выбираем один случайный прокси
    proxy = random.choice(proxies)
    session.proxies = {"http": proxy, "https": proxy}
    return session


print(get_free_proxies)

for i in range(5):
    
    #print(s.proxies)
    #proxy = random.choice(['91.187.69.101:4153'])
    
    proxxies = {"http": '37.204.95.22:80', "https": '185.61.77.134:8088'}
    html = requests.get('https://whoer.net/', proxies=proxxies)
    print(html)
    print("Страница запроса с IP:", html.text.strip())
    

<Response [200]>
<function get_free_proxies at 0x0000024A86CA0790>


ProxyError: HTTPSConnectionPool(host='whoer.net', port=443): Max retries exceeded with url: / (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000024A876FB640>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение')))

In [9]:
from ipywidgets import IntProgress
from IPython.display import display
import csv

prgBar = IntProgress(min = 0, max = 499) # Создаем прогрессбар
display(prgBar)


SSD_LIST = URLsFromPage()
    
with open("ssd_links.csv", mode="w", encoding='utf-8') as w_file:
    for link in SSD_LIST:
        file_writer = csv.writer(w_file, delimiter = ";", lineterminator="\r")
        file_writer.writerow([link])
    
    
    
        prgBar.value += 1
        

SSD_LIST

IntProgress(value=0, max=499)

['https://www.citilink.ru/product/ssd-nakopitel-kingston-a400-sa400s37-480g-480gb-2-5-sata-iii-420253/properties/',
 'https://www.citilink.ru/product/ssd-nakopitel-kingston-a400-sa400s37-240g-240gb-2-5-sata-iii-420251/properties/',
 'https://www.citilink.ru/product/nakopitel-ssd-digma-sata-iii-512gb-dgsr2512gs93t-run-s9-2-5-1651622/properties/',
 'https://www.citilink.ru/product/nakopitel-ssd-digma-sata-iii-256gb-dgsr2256gs93t-run-s9-2-5-1651620/properties/',
 'https://www.citilink.ru/product/nakopitel-ssd-kingston-pci-e-x4-500gb-snvs-500g-nv1-m-2-2280-1504847/properties/',
 'https://www.citilink.ru/product/nakopitel-ssd-kingspec-sata-iii-256gb-p3-256-2-5-1740428/properties/',
 'https://www.citilink.ru/product/ssd-nakopitel-a-data-ultimate-su650-asu650ss-240gt-r-240gb-2-5-sata-ii-1091578/properties/',
 'https://www.citilink.ru/product/nakopitel-ssd-digma-sata-iii-128gb-dgsr2128gy23t-run-y2-2-5-1651615/properties/',
 'https://www.citilink.ru/product/nakopitel-ssd-netac-sata-iii-256gb-nt